# Loading graph data

In [70]:
#!pip install pandas
#!pip install scipy
#!pip install networkx 
#!pip install scikit-learn

In [ ]:
from fractions import Fraction
import pandas as pd
from scipy.sparse import csr_matrix
import networkx as nx

First let's access the directory consolidated_data where the graph data are stored

In [25]:
import os
folder_path = "../consolidated_data"
print(os.listdir(folder_path))

['fGraph_edges_3.tex', 'fGraph_dials_4.tex', 'amplitudeCoefficients_7.tex', 'fGraph_edges_4.tex', 'amplitudeCoefficients_6.tex', 'fGraph_dials_6.tex', 'fGraph_nums_5.tex', 'fGraph_dials_9.tex', 'fGraph_edges_2.tex', 'amplitudeCoefficients_1.tex', 'fGraph_nums_6.tex', 'fGraph_dials_1.tex', 'amplitudeCoefficients_3.tex', 'fGraph_nums_10.tex', 'amplitudeCoefficients_2.tex', 'fGraph_nums_2.tex', 'fGraph_edges_8.tex', 'fGraph_edges_5.tex', 'fGraph_edges_1.tex', 'fGraph_edges_6.tex', 'fGraph_dials_10.tex', 'amplitudeCoefficients_8.tex', 'fGraph_edges_9.tex', 'amplitudeCoefficients_5.tex', 'consolidated_correlator_data.nb', 'fGraph_nums_3.tex', 'fGraph_nums_8.tex', 'fGraph_dials_2.tex', 'amplitudeCoefficients_4.tex', 'fGraph_dials_8.tex', 'fGraph_dials_3.tex', 'fGraph_nums_9.tex', 'fGraph_edges_7.tex', 'amplitudeCoefficients_9.tex', 'fGraph_nums_7.tex', 'fGraph_dials_5.tex', 'fGraph_nums_4.tex', 'fGraph_nums_1.tex', 'fGraph_dials_7.tex', 'fGraph_edges_10.tex', 'amplitudeCoefficients_8 .txt', 

Load amplitudes coefficients for a fixed number of loops

In [79]:
loop = "7"

## Import Correlator Coefficients

In [86]:
# Load the amplitude_coefficient_loop file which contain an array of the form {1,2,3,4,5}
# First we set the path to the file
path_file= "../consolidated_data/amplitudeCoefficients_" + loop +".tex"
# Open and read the file
with open(path_file, 'r') as f:
    content = f.read().strip()  # Read the file content and remove any leading/trailing whitespace

# Assume the content is in the format "{....}"
if content.startswith("{") and content.endswith("}"):
    # Remove the enclosing curly braces
    content = content[1:-1]
    # Split the string by commas to get individual number strings
    number_strings = content.split(',')
    # Convert each number string to a Fraction 
    coefs = [Fraction(num.strip()) for num in number_strings]

    print("Successfully loaded the amplitude coefficients")
else:
    print("File content is not in the expected format: {....}")

Successfully loaded the amplitude coefficients


In [87]:
# Multiply by the biggest denominator
max_denominator = max(num.denominator for num in coefs)
# Convert to integers
coefs = [num.numerator * (max_denominator // num.denominator) for num in coefs]
len(coefs)

220

In [88]:
# Generate dataframe and add colums "COEFFICENTS"
# Create a DataFrame with the coefficients
df = pd.DataFrame(coefs, columns=["COEFFICIENTS"])

In [89]:
sum(df.COEFFICIENTS ==0), sum(df.COEFFICIENTS !=0)

(93, 127)

## Import Edges

In [106]:
del df_edges
# Load the file "graph_edges_7.csv" containg the edges
path_file = "graph_edges_"+ loop +".csv"
# Read the CSV file into a DataFrame
df_edges = pd.read_csv(path_file)
# Display the first few rows of the DataFrame
df_edges


,DEN_EDGES,NUM_EDGES
0,"{{1, 3}, {1, 9}, {1, 10}, {1, 11}, {2, 4}, {2,...","{{3, 11}, {4, 9}, {5, 10}, {6, 11}, {7, 8}}"
1,"{{1, 3}, {1, 9}, {1, 10}, {1, 11}, {2, 4}, {2,...","{{3, 11}, {4, 9}, {5, 7}, {6, 11}, {8, 10}}"
2,"{{1, 3}, {1, 9}, {1, 10}, {1, 11}, {2, 4}, {2,...","{{3, 11}, {4, 11}, {5, 7}, {6, 8}, {9, 10}}"
3,"{{1, 3}, {1, 9}, {1, 10}, {1, 11}, {2, 4}, {2,...","{{3, 11}, {4, 11}, {5, 10}, {6, 8}, {7, 9}}"
4,"{{1, 3}, {1, 9}, {1, 10}, {1, 11}, {2, 4}, {2,...","{{3, 4}, {5, 11}, {6, 11}, {7, 8}, {9, 10}}"
...,...,...
215,"{{1, 2}, {1, 4}, {1, 5}, {1, 10}, {2, 3}, {2, ...","{{8, 9}, {10, 11}, {10, 11}}"
216,"{{1, 4}, {1, 6}, {1, 9}, {1, 11}, {2, 3}, {2, ...","{{8, 9}, {10, 11}, {10, 11}, {10, 11}}"
217,"{{1, 5}, {1, 6}, {1, 9}, {1, 11}, {2, 3}, {2, ...","{{10, 11}, {10, 11}}"
218,"{{1, 4}, {1, 6}, {1, 8}, {1, 10}, {2, 3}, {2, ...","{{7, 11}, {8, 11}, {9, 10}}"


## Old import edges

In [51]:
# First we set the path to the file
path_file= "../consolidated_data/fGraph_edges_" + loop +".tex"
# Open and read the file
with open(path_file, 'r') as f:
    # read one line from the file until the end
    content = f.read()
    # remove /n
    content = content.replace("\n", "")
    # remove the first { and last }
    content = content[1:-1]
    # split on }{
    content = content.split("}{")
    # transform each element into a list of tuples
    edges = [tuple(map(int, item.split(','))) for item in content]

In [55]:
len(edges), len(df)

(1432, 2709)

In [54]:
# Add DEN_EDGES column to the df
df['DEN_EDGES'] = edges

ValueError: Length of values (1432) does not match length of index (2709)